In [3]:
# Import modules
from ax.api.client import Client
from ax.api.configs import  RangeParameterConfig, ChoiceParameterConfig
import json
import pandas as pd
import os
from pathlib import Path

[WARNING 09-02 14:09:53] ax.service.utils.with_db_settings_base: Ax currently requires a sqlalchemy version below 2.0. This will be addressed in a future release. Disabling SQL storage in Ax for now, if you would like to use SQL storage please install Ax with mysql extras via `pip install ax-platform[mysql]`.


In [ ]:
# TODO modify config.json file to use Ax conventions for parameters
# TODO move the load_data function to utils folder
# TODO make the following functions
    # 1. read in excel spreadsheet and translate to JSON with variable names
    # 2. read in JSON and produce Ax parameters


In [ ]:

## function to import the data spreadsheet file
def load_data(file_path, **kwargs):
    """
    Smart data loader with custom parameters for each file type.
    """
    file_path = Path(file_path)
    file_extension = file_path.suffix.lower()
    
    if file_extension in ['.xlsx', '.xls']:
        # Excel-specific parameters
        excel_kwargs = {k: v for k, v in kwargs.items() 
                       if k in ['sheet_name', 'header', 'skiprows']}
        return pd.read_excel(file_path, **excel_kwargs)
        
    elif file_extension == '.csv':
        # CSV-specific parameters
        csv_kwargs = {k: v for k, v in kwargs.items() 
                     if k in ['header', 'index_col', 'sep', 'delimiter']}
        return pd.read_csv(file_path, **csv_kwargs)
    

In [10]:
#Define file path here 
file_path = Path.cwd() / "examples" / "data" / "BDFO film analysis_2025-04-23.xlsx"

df = load_data(file_path, skiprows=2)
## print the first few rows to verify the data was loaded correctly
# df.head()

In [11]:
# Rename columns for readability
df.rename(columns={
        'Sample\n#': 'sample_number',
        'Split target\n#': 'split_target_number',
        'Single composition target\n#': 'single_composition_target_number',
        'Target conditioning time (min)': 'target_conditioning_time_min',
        'Deposition time (min)' : 'deposition_time_min',
        'Target rotation (rpm)' : 'target_rotation_rpm',
        'Fluence\n(J/cm2)' : 'fluence_J_per_cm2',
        'Substrate' : 'substrate_material',
        'Layer height (mm)' : 'layer_height_mm',
        'Target to subsrate distance (cm)' : 'target_to_substrate_distance_cm',
        'Date deposited' : 'date_deposited',
        'T (C)' : 'temperature_C',
        'O2 pressure (mTorr)' : 'O2_pressure_mTorr',
        'Raster speed ratio\n(A:B)' : 'raster_speed_ratio_AtoB',
        'Fraction of time on side A' : 'fraction_of_time_on_side_A',

    }, inplace=True)


# data type conversion [not needed in current iteration]

# df.astype({
#         'sample_number': 'string',
#         'split_target_number': 'string',
#         'single_composition_target_number': 'string'
#     })
    

pd.set_option('display.max_columns', None)
df.head()

,sample_number,split_target_number,single_composition_target_number,target_conditioning_time_min,deposition_time_min,target_rotation_rpm,fluence_J_per_cm2,substrate_material,target_to_substrate_distance_cm,date_deposited,temperature_C,O2_pressure_mTorr,raster_speed_ratio_AtoB,fraction_of_time_on_side_A,Bi,Dy,Fe,O,Bi.1,Dy.1,Fe.1,Date measured,Date analyzed,Standards,Elements,Bi.2,Dy.2,Fe.2,O.1,Bi.3,Dy.3,Fe.3
0,PLD041,TGT50,NaN,0,62,10,NaN,Si,5.9,2016-08-11,25.0,0.3,1.0,0.5,24.96,9.48,5.56,60.0,62.4,23.7,13.9,2018-01-02,2018-01-02,"Si, Bi, DyPO4, Fe, MgO",Bi Dy Fe O,29.08,9.82,1.58,59.52,71.837945,24.258893,3.903162
1,PLD048,TGT50,NaN,0,60,10,NaN,Si,5.9,2016-08-12,690.0,0.3,1.0,0.5,24.96,9.48,5.56,60.0,62.4,23.7,13.9,2018-01-02,2018-01-02,"Si, Bi, DyPO4, Fe, MgO",Bi Dy Fe O,15.31,22.29,4.01,58.39,36.794040,53.568854,9.637106
2,PLD053,TGT50,NaN,0,60,10,NaN,Si,5.9,2016-10-07,690.0,0.3,1.0,0.5,24.96,9.48,5.56,60.0,62.4,23.7,13.9,2018-01-02,2018-01-02,"Si, Bi, DyPO4, Fe, MgO",Bi Dy Fe O,9.35,19.93,4.71,66.02,27.508091,58.634893,13.857017
3,PLD061,TGT50,NaN,0,60,0.006,NaN,Si,5.9,2017-05-17,25.0,0.3,1.0,0.5,24.96,9.48,5.56,60.0,62.4,23.7,13.9,2018-01-02,2018-01-02,"Si, Bi, DyPO4, Fe, MgO",Bi Dy Fe O,25.61,10.15,3.23,61.01,65.683509,26.032316,8.284175
4,PLD065,TGT50,NaN,0,60,0.005,NaN,Si,5.9,2017-05-18,690.0,0.3,1.0,0.5,24.96,9.48,5.56,60.0,62.4,23.7,13.9,2018-01-02,2018-01-02,"Si, Bi, DyPO4, Fe, MgO",Bi Dy Fe O,11.69,24.43,4.64,59.24,28.680079,59.936212,11.383710


In [8]:
# read in the config file and set up parameters

# Bounds and configs are loaded in config file (config.json)
configPath = 'config.json'

# Load the full config data file
try:
    # Load the full config data file
    with open(configPath, 'r') as f:
        config = json.load(f)
        print(f"Loaded config: file from {configPath}")

    # Extract bounds for parameters
    pbounds = config['pbounds']

    ## Other parameters that can be loaded via config
    # optimizer_settings = config['optimizer_settings']
    # model_params = config['model_parameters']

    # Print usage
    print("Loaded config parameters:")
    print(json.dumps(config, indent=4))


except FileNotFoundError:
    print(f"Error: Could not find the config file at '{configPath}'. Please check the path and try again.")

except json.JSONDecodeError as e:
    print(f"Error: Failed to parse JSON in config file '{configPath}': {e}")

except Exception as e:
    print(f"An unexpected error occurred while loading the config file: {e}")



Loaded config: file from config.json
Loaded config parameters:
{
    "pbounds": {
        "zStagePosition": [
            0,
            51
        ],
        "flowRate": [
            0,
            10
        ],
        "pressure": [
            0,
            125
        ],
        "heaterSetpoint": [
            0,
            1000
        ],
        "substrateRotation": [
            0,
            30
        ],
        "targetRotation": [
            0,
            30
        ],
        "depositionTime": [
            0,
            1000
        ],
        "laserEnergy": [
            0,
            500
        ],
        "laserSpotSize": [
            0,
            5
        ],
        "laserRepetitionRate": [
            0,
            100
        ],
        "coolingTime": [
            0,
            1000
        ]
    },
    "optimizer_settings": {
        "init_points": 5,
        "n_iter": 25,
        "acq": "ei"
    },
    "model_parameters": {
        "learning_rate": 0.

In [ ]:
infill_density = RangeParameterConfig(name="infill_density", parameter_type="float", bounds=(0,100))
layer_height = RangeParameterConfig(name="layer_height", parameter_type="float", bounds=(0.1,0.4))
infill_type = ChoiceParameterConfig(name="infill_type", parameter_type="str", values=["honeycomb", "gyroid", "lines", "rectilinear"])

client.configure_experiment(
    parameters=[infill_density, layer_height, infill_type],
    # the following areguments are only necessary when saving to the DB
    name="3D_print_strength_experiment",
    description="Maximize the strength of the 3D printed parts",
    owner="developer",
    )